In [ ]:
%pip install -q "git+https://github.com/huggingface/optimum-intel.git" 
%pip install -q "nncf>=2.6.0"
%pip install -q "gradio"
%pip install -q "onnx" "onnxruntime" "einops" "transformers>=4.31.0"
%pip install -q "openvino>=2023.1.0"

In [6]:
import ipywidgets as widgets

import torch

#from transformers import AutoModelForCausalLM
from optimum.intel.openvino import OVModelForCausalLM

from transformers import AutoTokenizer, pipeline

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


No CUDA runtime is found, using CUDA_HOME='C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.2'


In [55]:
from pathlib import Path
from config import read_config

system_config = read_config()

#--- Contents from 'config.py'
def llama_partial_text_processor(partial_text, new_text):
    new_text = new_text.replace("[INST]", "").replace("[/INST]", "")
    partial_text += new_text
    return partial_text

#DEFAULT_SYSTEM_PROMPT = """\
#You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
#If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\
#"""

DEFAULT_SYSTEM_PROMPT_JP = """\
あなたは親切で、礼儀正しく、誠実なアシスタントです。 常に安全を保ちながら、できるだけ役立つように答えてください。 回答には、有害、非倫理的、人種差別的、性差別的、有毒、危険、または違法なコンテンツを含めてはいけません。 回答は社会的に偏見がなく、本質的に前向きなものであることを確認してください。
質問が意味をなさない場合、または事実に一貫性がない場合は、正しくないことに答えるのではなく、その理由を説明してください。 質問の答えがわからない場合は、誤った情報を共有しないでください。\
"""

#model_id = { "youri-7b-chat" : { 
#                "model_id":"rinna/youri-7b-chat",
#                "start_message": f"<s>[INST] <<SYS>>\n{DEFAULT_SYSTEM_PROMPT }\n<</SYS>>\n\n",
#                "history_template": "{user}[/INST]{assistant}</s><s>",
#                "current_message_template": "{user} [/INST]{assistant}",
#                "tokenizer_kwargs": {"add_special_tokens":False},
#                "partial_text_processor": llama_partial_text_processor,
#                "revision": "5514c85fedd6c4fc0fc66fa533bc157de520da73"}}

model_id = { "youri-7b-chat" : { 
                "model_id":"rinna/youri-7b-chat",
                "start_message": f"設定: {DEFAULT_SYSTEM_PROMPT_JP}\n",
                "history_template": "ユーザー: {user}\nシステム: {assistant}\n",
                "current_message_template": "ユーザー: {user}\nシステム: {assistant}",
                "tokenizer_kwargs": {"add_special_tokens":False},
                "partial_text_processor": llama_partial_text_processor,
                "revision": "5514c85fedd6c4fc0fc66fa533bc157de520da73"}}
#---

model_name = "youri-7b-chat"

fp16_model_dir = Path(model_name) / "FP16"
compressed_model_dir = Path(model_name) / "INT8_compressed_weights"
model_dir = Path(model_name) / "FP16"
pt_model_id = model_configuration["model_id"]
cache_dir = system_config['hf_cache_dir']

In [56]:
CONVERT_FP16 = False
CONVERT_INT8 = True
model_configuration = model_id[model_name]
print(model_configuration)

{'model_id': 'rinna/youri-7b-chat', 'start_message': '設定: あなたは親切で、礼儀正しく、誠実なアシスタントです。 常に安全を保ちながら、できるだけ役立つように答えてください。 回答には、有害、非倫理的、人種差別的、性差別的、有毒、危険、または違法なコンテンツを含めてはいけません。 回答は社会的に偏見がなく、本質的に前向きなものであることを確認してください。\n質問が意味をなさない場合、または事実に一貫性がない場合は、正しくないことに答えるのではなく、その理由を説明してください。 質問の答えがわからない場合は、誤った情報を共有しないでください。\n', 'history_template': 'ユーザー: {user}\nシステム: {assistant}\n', 'current_message_template': 'ユーザー: {user}\nシステム: {assistant}', 'tokenizer_kwargs': {'add_special_tokens': False}, 'partial_text_processor': <function llama_partial_text_processor at 0x0000024E8FC7F0A0>, 'revision': '5514c85fedd6c4fc0fc66fa533bc157de520da73'}


## Instantiate Model using Optimum Intel [$\uparrow$](#Table-of-content:)

In [4]:
from optimum.intel import OVQuantizer
from transformers import AutoModelForCausalLM
from optimum.intel.openvino import OVModelForCausalLM
import logging
import nncf
import gc

nncf.set_log_level(logging.ERROR)

"""
if CONVERT_INT8 and not compressed_model_dir.exists():
    pt_model = AutoModelForCausalLM.from_pretrained(pt_model_id)
    quantizer = OVQuantizer.from_pretrained(pt_model)
    quantizer.quantize(save_directory=compressed_model_dir, weights_only=True)
    del quantizer
    del pt_model
    gc.collect()

if CONVERT_FP16 and not model_dir.exists():
    ov_model = OVModelForCausalLM.from_pretrained(pt_model_id, export=True, compile=False)
    ov_model.half()
    ov_model.save_pretrained(model_dir)
    del ov_model
"""
gc.collect();


In [5]:
pt_model = AutoModelForCausalLM.from_pretrained(pt_model_id, cache_dir=cache_dir)

In [ ]:
#torch.save(pt_model, "youri-7b-chat.pth")

In [6]:
quantizer = OVQuantizer.from_pretrained(pt_model)

In [8]:
quantizer.quantize(save_directory=compressed_model_dir, weights_only=True)

No configuration describing the quantization process was provided, a default OVConfig will be generated.
Using framework PyTorch: 1.13.1+cpu
Overriding 1 configuration item(s)
	- use_cache -> True
N:\work\venv-ov23.1\lib\site-packages\transformers\models\llama\modeling_llama.py:808: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_shape[-1] > 1:
N:\work\venv-ov23.1\lib\site-packages\transformers\models\llama\modeling_llama.py:79: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if past_key_values_length > 0:
N:\work\venv-ov23.1\lib\site-packages\tra

RuntimeError: bad allocation

In [9]:
del quantizer

In [10]:
del pt_model

In [ ]:
ov_model = OVModelForCausalLM.from_pretrained(pt_model_id, export=True, compile=False, cache_dir=cache_dir)

In [ ]:
ov_model.half()

In [ ]:
fp16_model_dir = Path(model_name) / "FP16"
ov_model.save_pretrained(fp16_model_dir)

In [ ]:
del ov_model

In [ ]:
gc.collect()

### Weights Compression using NNCF [$\uparrow$](#Table-of-content:)

In [11]:
#compressed_model_dir = Path(model_id.value) / "INT8_compressed_weights"
model_dir = Path(model_id.value) / "FP16"

gc.collect();

AttributeError: 'dict' object has no attribute 'value'

In [8]:
fp16_weights = model_dir / "openvino_model.bin"
#int8_weights = compressed_model_dir / "openvino_model.bin"

if fp16_weights.exists():
    print(f'Size of FP16 model in MB is {fp16_weights.stat().st_size / 1024 / 1024}')

Size of FP16 model in MB is 12854.508113861084


In [9]:
import openvino as ov
core = ov.Core()

device = system_config['device']

In [12]:
import warnings
from pathlib import Path
from optimum.intel.openvino import OVModelForCausalLM
from transformers import AutoTokenizer, AutoConfig

model_dir = Path(model_name) / "FP16"

ov_config = {'PERFORMANCE_HINT': 'LATENCY', 'NUM_STREAMS': '1', "CACHE_DIR": ""}

tok = AutoTokenizer.from_pretrained(model_configuration["model_id"], trust_remote_code=True)

model_class = OVModelForCausalLM
ov_model = model_class.from_pretrained(model_dir, device=device, ov_config=ov_config, config=AutoConfig.from_pretrained(model_dir, trust_remote_code=True), trust_remote_code=True, cache_dir=cache_dir)

The argument `trust_remote_code` is to be used along with export=True. It will be ignored.
Compiling the model to CPU ...


In [34]:
# Sample prompt from a Youri sample program
'''
prompt = """設定: 次の日本語を英語に翻訳してください。
ユーザー: 自然言語による指示に基づきタスクが解けるよう学習させることを Instruction tuning と呼びます。
システム: """
'''

prompt = """ユーザー: 日本で二番目に高い山は何ですか？
システム: """

input_tokens = tok(prompt, return_tensors="pt", add_special_tokens=False)
answer = ov_model.generate(**input_tokens, max_new_tokens=50)
#answer = ov_model.generate(**input_tokens)
print(tok.batch_decode(answer)[0])

ユーザー: 日本で二番目に高い山は何ですか？
システム:  日本で二番目に高い山は、槍ヶ岳（3,180m）です。</s>


In [58]:
from threading import Event, Thread
from uuid import uuid4
from typing import Optional, Union, Dict, Tuple, List

import gradio as gr
import torch
from transformers import (
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    TextIteratorStreamer,
)

model_name               = model_configuration["model_id"]
history_template         = model_configuration["history_template"]
current_message_template = model_configuration["current_message_template"]
start_message            = model_configuration["start_message"]
stop_tokens              = model_configuration.get("stop_tokens")
tokenizer_kwargs         = model_configuration.get("tokenizer_kwargs", {})



max_new_tokens = 256

class StopOnTokens(StoppingCriteria):
    def __init__(self, token_ids):
        self.token_ids = token_ids
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in self.token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

if stop_tokens is not None:
    if isinstance(stop_tokens[0], str):
        stop_tokens = tok.convert_tokens_to_ids(stop_tokens)
        
    stop_tokens = [StopOnTokens(stop_tokens)]

def default_partial_text_processor(partial_text:str, new_text:str):
    """
    helper for updating partially generated answer, used by de
    
    Params:
      partial_text: text buffer for storing previosly generated text
      new_text: text update for the current step
    Returns:
      updated text string
    
    """
    partial_text += new_text
    return partial_text

text_processor = model_configuration.get("partial_text_processor", default_partial_text_processor)
    
def convert_history_to_text(history:List[Tuple[str, str]]):
    """
    function for conversion history stored as list pairs of user and assistant messages to string according to model expected conversation template
    Params:
      history: dialogue history
    Returns:
      history in text format
    """
    text = start_message + "".join(
        [
            "".join(
                [
                    history_template.format(user=item[0], assistant=item[1])
                ]
            )
            for item in history[:-1]
        ]
    )
    text += "".join(
        [
            "".join(
                [
                    current_message_template.format(user=history[-1][0], assistant=history[-1][1])
                ]
            )
        ]
    )
    return text



def user(message, history):
    """
    callback function for updating user messages in interface on submit button click
    
    Params:
      message: current message
      history: conversation history
    Returns:
      None
    """
    # Append the user's message to the conversation history
    return "", history + [[message, ""]]


def bot(history, temperature, top_p, top_k, repetition_penalty, conversation_id):
    """
    callback function for running chatbot on submit button click
    
    Params:
      history: conversation history
      temperature:  parameter for control the level of creativity in AI-generated text. 
                    By adjusting the `temperature`, you can influence the AI model's probability distribution, making the text more focused or diverse.
      top_p: parameter for control the range of tokens considered by the AI model based on their cumulative probability.
      top_k: parameter for control the range of tokens considered by the AI model based on their cumulative probability, selecting number of tokens with highest probability.
      repetition_penalty: parameter for penalizing tokens based on how frequently they occur in the text.
      conversation_id: unique conversation identifier.
    
    """

    # Construct the input message string for the model by concatenating the current system message and conversation history
    messages = convert_history_to_text(history)

    # Tokenize the messages string
    input_ids = tok(messages, return_tensors="pt", **tokenizer_kwargs).input_ids
    if input_ids.shape[1] > 2000:
        history = [history[-1]]
        messages = convert_history_to_text(history)
        input_ids = tok(messages, return_tensors="pt", **tokenizer_kwargs).input_ids
    streamer = TextIteratorStreamer(tok, timeout=30.0, skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        input_ids=input_ids,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=temperature > 0.0,
        top_p=top_p,
        top_k=top_k,
        repetition_penalty=repetition_penalty,
        streamer=streamer,
    )
    if stop_tokens is not None:
        generate_kwargs["stopping_criteria"] = StoppingCriteriaList(stop_tokens)

    stream_complete = Event()

    def generate_and_signal_complete():
        """
        genration function for single thread
        """
        global start_time
        ov_model.generate(**generate_kwargs)
        stream_complete.set()

    t1 = Thread(target=generate_and_signal_complete)
    t1.start()

    # Initialize an empty string to store the generated text
    partial_text = ""
    for new_text in streamer:
        partial_text = text_processor(partial_text, new_text)
        history[-1][1] = partial_text
        yield history


def get_uuid():
    """
    universal unique identifier for thread
    """
    return str(uuid4())


with gr.Blocks(
    theme=gr.themes.Soft(),
    css=".disclaimer {font-variant-caps: all-small-caps;}",
) as demo:
    conversation_id = gr.State(get_uuid)
    gr.Markdown(
        f"""<h1><center>OpenVINO {model_name} Chatbot</center></h1>"""
    )
    chatbot = gr.Chatbot(height=500)
    with gr.Row():
        with gr.Column():
            msg = gr.Textbox(
                label="Chat Message Box",
                placeholder="Chat Message Box",
                show_label=False,
                container=False
            )
        with gr.Column():
            with gr.Row():
                submit = gr.Button("Submit")
                stop = gr.Button("Stop")
                clear = gr.Button("Clear")
    with gr.Row():
        with gr.Accordion("Advanced Options:", open=False):
            with gr.Row():
                with gr.Column():
                    with gr.Row():
                        temperature = gr.Slider(
                            label="Temperature",
                            value=0.1,
                            minimum=0.0,
                            maximum=1.0,
                            step=0.1,
                            interactive=True,
                            info="Higher values produce more diverse outputs",
                        )
                with gr.Column():
                    with gr.Row():
                        top_p = gr.Slider(
                            label="Top-p (nucleus sampling)",
                            value=1.0,
                            minimum=0.0,
                            maximum=1,
                            step=0.01,
                            interactive=True,
                            info=(
                                "Sample from the smallest possible set of tokens whose cumulative probability "
                                "exceeds top_p. Set to 1 to disable and sample from all tokens."
                            ),
                        )
                with gr.Column():
                    with gr.Row():
                        top_k = gr.Slider(
                            label="Top-k",
                            value=50,
                            minimum=0.0,
                            maximum=200,
                            step=1,
                            interactive=True,
                            info="Sample from a shortlist of top-k tokens — 0 to disable and sample from all tokens.",
                        )
                with gr.Column():
                    with gr.Row():
                        repetition_penalty = gr.Slider(
                            label="Repetition Penalty",
                            value=1.1,
                            minimum=1.0,
                            maximum=2.0,
                            step=0.1,
                            interactive=True,
                            info="Penalize repetition — 1.0 to disable.",
                        )
    gr.Examples([
        ["Hello there! How are you doing?"],
        ["What is OpenVINO?"],
        ["Who are you?"],
        ["Can you explain to me briefly what is Python programming language?"],
        ["Explain the plot of Cinderella in a sentence."],
        ["What are some common mistakes to avoid when writing code?"],
        ["Write a 100-word blog post on “Benefits of Artificial Intelligence and OpenVINO“"]
    ], 
        inputs=msg, 
        label="Click on any example and press the 'Submit' button"
    )

    submit_event = msg.submit(
        fn=user,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot],
        queue=False,
    ).then(
        fn=bot,
        inputs=[
            chatbot,
            temperature,
            top_p,
            top_k,
            repetition_penalty,
            conversation_id,
        ],
        outputs=chatbot,
        queue=True,
    )
    submit_click_event = submit.click(
        fn=user,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot],
        queue=False,
    ).then(
        fn=bot,
        inputs=[
            chatbot,
            temperature,
            top_p,
            top_k,
            repetition_penalty,
            conversation_id,
        ],
        outputs=chatbot,
        queue=True,
    )
    stop.click(
        fn=None,
        inputs=None,
        outputs=None,
        cancels=[submit_event, submit_click_event],
        queue=False,
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.queue(max_size=2)
# if you are launching remotely, specify server_name and server_port
#  demo.launch(server_name='your server name', server_port='server port in int')
# if you have any issue to launch on your platform, you can pass share=True to launch method:
# demo.launch(share=True)
# it creates a publicly shareable link for the interface. Read more in the docs: https://gradio.app/docs/
demo.launch()

N:\work\venv-ov23.1\lib\site-packages\gradio\blocks.py:891: UserWarning: api_name user already exists, using user_1
  warnings.warn(f"api_name {api_name} already exists, using {api_name_}")
N:\work\venv-ov23.1\lib\site-packages\gradio\blocks.py:891: UserWarning: api_name bot already exists, using bot_1
  warnings.warn(f"api_name {api_name} already exists, using {api_name_}")


Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


N:\work\venv-ov23.1\lib\site-packages\optimum\intel\openvino\modeling_decoder.py:388: FutureWarning: `shared_memory` is deprecated and will be removed in 2024.0. Value of `shared_memory` is going to override `share_inputs` value. Please use only `share_inputs` explicitly.
  self.request.start_async(inputs, shared_memory=True)
